## Task 2:
Code provided by the professor on course website

In [384]:
#!/usr/bin/env python

# Written by Chris Conly based on C++
# code provided by Vassilis Athitsos
# Written to be Python 2.4 compatible for omega

from copy import copy
import random
import sys
import math
import itertools

    

class node:
    def __init__(self, val = 0, alpha = -math.inf, beta = math.inf, max_flag = 1, pred = None, child = [], moves = []):
        self.val = val
        self.alpha = alpha
        self.beta = beta
        self.max_flag = max_flag
        self.pred = pred
        self.child = child
        self.moves = moves
        
    def create_child(self, branch_factor, depth, pred):
        if depth == 0:
            return
        if depth == 2:
            for i in range(branch_factor):
                child1 = node(pred = pred, child = None)
                pred.child.append(child1)      # Undo this change if wrong
            #self.create_child(branch_factor, depth-1, child1)
            print("Node:", child1)
            print("Printing children", child1.child)
            print("Pred:", child1.pred)
            return
        #max_flag = 1
        for i in range(branch_factor):
            child1 = node(pred = pred, child = [])
            pred.child.append(child1)
            self.create_child(branch_factor, depth-1, child1)
            #max_flag += 1
            #max_flag = max_flag % 2
        print("Node:", child1)
        print("Printing children", child1.child)
        print("Pred:", child1.pred)
        return child1
    

class maxConnect4Game:
    def __init__(self):
        self.gameBoard = [[0 for i in range(7)] for j in range(6)]
        self.currentTurn = 1
        self.player1Score = 0
        self.player2Score = 0
        self.pieceCount = 0
        self.gameFile = None
        random.seed()

    

    # Count the number of pieces already played
    def checkPieceCount(self):
        self.pieceCount = sum(1 for row in self.gameBoard for piece in row if piece)

    # Output current game status to console
    def printGameBoard(self):
        print (' -----------------')
        for i in range(6):
            print (' |', end = "")
            for j in range(7):
                print(self.gameBoard[i][j], end = "")
            print ('| ')
        print (' -----------------')
    
    # Output current game status to console
    def printGameBoard_pass(self, board):
        print (' -----------------')
        for i in range(6):
            print (' |', end = "")
            for j in range(7):
                print(board[i][j], end = "")
            print ('| ')
        print (' -----------------')

    
    # Output current game status to file
    def printGameBoardToFile(self):
        for row in self.gameBoard:
            self.gameFile.write(''.join(str(col) for col in row) + '\r\n')
        self.gameFile.write('%s\r\n' % str(self.currentTurn))

    # Place the current player's piece in the requested column
    def playPiece(self, column):
        if self.gameBoard[0][column] == 0:
            for i in range(5, -1, -1):
                if not self.gameBoard[i][column]:
                    self.gameBoard[i][column] = self.currentTurn
                    self.pieceCount += 1
                    return 1
        return -1
                
    def playPiece_new(self, column, board, currentTurn):
        #print(column)
        #print("Printing from playPiece_new function:\n", board)
        if board[0][column] == 0:
            for i in range(5, -1, -1):
                if board[i][column] == 0:
                    board[i][column] = currentTurn
                    #self.pieceCount += 1
                    return board, 0
        return board, -1

    #def eval_func(self):
    #    return random.randrange(0,7)
    
    # The AI section. Currently plays randomly.
    def aiPlay(self):
        # Add code here to select the column after move
        #print("Evaluation score of the current board setup:", self.eval_func(self.gameBoard))
        print("AI is thinking...")
        randColumn = self.game_minimax()
        print("Column selected by AI:", randColumn)
        #randColumn = random.randrange(0,7)
        result = self.playPiece(randColumn-1)######Durva
        if not result:
            self.aiPlay()
        else:
            print('\n\nmove %d: Player %d, column %d\n' % (self.pieceCount, self.currentTurn, randColumn+1))
            if self.currentTurn == 1:
                self.currentTurn = 2
            elif self.currentTurn == 2:
                self.currentTurn = 1

    def get_minmax(self, temp_main, max_flag, obj):
        temp = []
        for i in temp_main:
                if i.val != '#':
                    temp.append(i)
                    if max_flag == 1:
                        if i.val > obj.alpha:
                            obj.maxim = i.val
                            obj.alpha = i.val
                            
                    
        if max_flag == 1:
            if temp != []:
                return max(temp)
            else:
                return '#'
        else:
            if temp != []:
                return min(temp)
            else:
                return '#'
    
    #def create_child(self, branch_factor, depth):
    #    if depth == 0:
    #        return
    #    max_flag = 1
    #    for i in range(branch_factor):
    #        if i == 0:
    #            child = node(max_flag = max_flag, moves = [])
    #        else:
    #            child = node(max_flag = max_flag, moves = x.moves.append(i))
    #        child.child.append(child)
    #        child.create_child(branch_factor, depth-1)
    #        
    #        max_flag += 1
    #        max_flag = max_flag % 2
    #    
    #    return child
    
    def alpha_beta_pruning(self, branch_factor, depth, root):
        current = root
        if current == None:
            return
        
        stack = []
        print("In alpha beta pruning function")
        stack.append(current)
        
        moves = []
        for i in range(1, branch_factor + 1):
            moves.append(i)
        print("available moves options: ", moves)
        comb = []
        for i in range(depth-1):
            comb.append(moves)
        p_moves = []
        for t in itertools.product(*comb):
            p_moves.append(list(t))
        print("All possible moves: \n", p_moves)
        moves_count = 0
        while (stack):
            top = stack.pop()
            if top.pred != None:
                top.alpha = top.pred.alpha
                top.beta = top.pred.beta
                top.max_flag = (top.pred.max_flag + 1) % 2
                #top.max_flag = top.max_flag % 2
            else:
                top.max_flag = 1
            print("Current node:", top)
            print("Printing pred and child nodes:")
            if top.child == None:
                # Here you will find leaf nodes from left to right
                print("Leaf node:", top)
                gcopy = [row[:] for row in self.gameBoard]   #Copy current game board
                ct = (self.currentTurn + 1) % 2
        
                for m in p_moves[moves_count]:
                    ct = (self.currentTurn + 1) % 2
                    gcopy, check_flag = self.playPiece_new(j-1, gcopy, ct) # Play current player move
                    # if move not possible then above function returns -1 and thus break if move not possible
                    if check_flag == -1:
                        break
                # the move was not possible then eval score = -1
                if check_flag == -1:
                    top.val = '#'
                    check_flag = 0
                else:
                    top.val = self.eval_func(gcopy)
                moves_count += 1
            
            else:
                for child in top.child:
                    # Here DFS traversal is happening
                    print(f"Pred of {child}", child.pred)
                    stack.append(child)
                    if child.max_flag = 0: #min node
                        if child.val > top.alpha:
                            stack.append()
            
            

    def game_minimax(self):
        # set the depth of minimax tree and branch factor = number of columns
        depth = 3
        branch_factor = 7
        #root = node()
        create_child = node().create_child
        root = node(pred = None)
        create_child(branch_factor, depth, root)
        print("Root:", root)
        print("Printing children", root.child)
        print("Pred:", root.pred)
        self.alpha_beta_pruning(branch_factor, depth, root)
        
    
    
        
        # Create array of empty arrays of size = depth
        #a = []
        #for i in range(depth):
        #    a.append([])
        ##print(a)
        
        # Create different levels combinations
        #level = []
        #for i in range(depth-1):
        #    level.append(1)
        
        # Some preset variables
        #index1 = len(level) - 1
        #index2 = len(a) - 1
        
        # Play all possible moves and accumulate the eval scores in last column of array a
        #while sum(level) < (depth+1)*branch_factor:
        #    #check if level matrix is valid and update accordingly
        #    for i in range(len(level)-1, 0, -1):
        #        if level[i] > branch_factor:
        #            level[i] = 1
        #            level[i-1] += 1
        #    if level[0] > branch_factor:
        #        break
        #    
        #    # play 10 moves depending upon level matrix
        #    # if set of moves not possible then write score as -1
        #    game_eval = random.randint(0,5)
        #    gcopy = [row[:] for row in self.gameBoard]   #Copy current game board
        #    ct = self.currentTurn
        #    # Change current player
        #    if ct == 1:
        #        ct = 2
        #    elif ct == 2:
        #        ct = 1
        #    
        #    #print("Moves:", level)
        #    for j in level:
        #        # Change current player
        #        if ct == 1:
        #            ct = 2
        #        elif ct == 2:
        #            ct = 1
        #            
        #        # Play the game with current player
        #        gcopy, check_flag = self.playPiece_new(j-1, gcopy, ct)
        #        
        #        # if move not possible then above function returns -1 and thus break if move not possible
        #        if check_flag == -1:
        #            break
        #    
        #    # the move was not possible then eval score = -1
        #    if check_flag == -1:
        #        game_eval = '#'
        #        check_flag = 0
        #    else:
        #        game_eval = self.eval_func(gcopy)
        #    #######################################
        #    #self.printGameBoard_pass(gcopy)
        #    #print("Eval score:", game_eval)
        #    #######################################
        #    
        #    #*************** change game_eval to object with alpha, beta, maximizer and value
        #    obj = node(game_val, -math.inf, math.inf)
        #    a[index2].append(obj)
        #    #a[index2].append(game_eval)
        #    level[index1] += 1
        #    #print(a)
        #        
        #    
        ##print(level)
        ##print(a)
        ##print(len(a[len(a)-1]))
        #
        ## if depth even then start with min (i.e. max_flag = 0) or else max (i.e. max_flag = 1)
        #max_flag = (depth) % 2
        #temp = []
        #c = 0
        #
        ##print(a)
        #for k in range(1,len(level)+1):
        #    max_flag += 1
        #    max_flag = max_flag % 2
        #    for i in range(len(a[len(a)-k])):
        #        if c == 0 or c%branch_factor != 0:
        #            temp.append(a[len(a)-k][i])
        #        else:
        #            if len(temp) == branch_factor:
        #                #********************Create object and add it to array (alpha, beta, maximizer, value)
        #                maxim = -math.inf if max_flag == 1 else maxim = math.inf
        #                obj = node(0, -math.inf, math.inf, maxim)
        #                obj = self.get_minmax(temp, max_flag, obj)
        #                #a[len(a)-k-1].append(self.get_minmax(temp, max_flag))
        #            temp = []
        #            temp.append(a[len(a)-k][i])
        #        c += 1
        #    #if max_flag == 1 and len(temp) == branch_factor:
        #    #    a[len(a)-k-1].append(max(temp))
        #    #elif max_flag == 0 and len(temp) == branch_factor:
        #    #    a[len(a)-k-1].append(min(temp))
        #    if len(temp) == branch_factor:
        #        a[len(a)-k-1].append(self.get_minmax(temp, max_flag))
        #    c = 0
        #    temp = []
        ##print(a)
        #
        #return (a[1].index(a[0][0])) + 1

    # Calculate the number of 4-in-a-row each player has
    def countScore_custom(self, board):
        player1Score = 0;
        player2Score = 0;

        # Check horizontally
        for row in board:
            # Check player 1
            if row[0:4] == [1]*4:
                player1Score += 1
            if row[1:5] == [1]*4:
                player1Score += 1
            if row[2:6] == [1]*4:
                player1Score += 1
            if row[3:7] == [1]*4:
                player1Score += 1
            # Check player 2
            if row[0:4] == [2]*4:
                player2Score += 1
            if row[1:5] == [2]*4:
                player2Score += 1
            if row[2:6] == [2]*4:
                player2Score += 1
            if row[3:7] == [2]*4:
                player2Score += 1

        # Check vertically
        for j in range(7):
            # Check player 1
            if (board[0][j] == 1 and board[1][j] == 1 and
                   board[2][j] == 1 and board[3][j] == 1):
                player1Score += 1
            if (board[1][j] == 1 and board[2][j] == 1 and
                   board[3][j] == 1 and board[4][j] == 1):
                player1Score += 1
            if (board[2][j] == 1 and board[3][j] == 1 and
                   board[4][j] == 1 and board[5][j] == 1):
                player1Score += 1
            # Check player 2
            if (board[0][j] == 2 and board[1][j] == 2 and
                   board[2][j] == 2 and board[3][j] == 2):
                player2Score += 1
            if (board[1][j] == 2 and board[2][j] == 2 and
                   board[3][j] == 2 and board[4][j] == 2):
                player2Score += 1
            if (board[2][j] == 2 and board[3][j] == 2 and
                   board[4][j] == 2 and board[5][j] == 2):
                player2Score += 1

        # Check diagonally

        # Check player 1
        if (board[2][0] == 1 and board[3][1] == 1 and
               board[4][2] == 1 and board[5][3] == 1):
            player1Score += 1
        if (board[1][0] == 1 and board[2][1] == 1 and
               board[3][2] == 1 and board[4][3] == 1):
            player1Score += 1
        if (board[2][1] == 1 and board[3][2] == 1 and
               board[4][3] == 1 and board[5][4] == 1):
            player1Score += 1
        if (board[0][0] == 1 and board[1][1] == 1 and
               board[2][2] == 1 and board[3][3] == 1):
            player1Score += 1
        if (board[1][1] == 1 and board[2][2] == 1 and
               board[3][3] == 1 and board[4][4] == 1):
            player1Score += 1
        if (board[2][2] == 1 and board[3][3] == 1 and
               board[4][4] == 1 and board[5][5] == 1):
            player1Score += 1
        if (board[0][1] == 1 and board[1][2] == 1 and
               board[2][3] == 1 and board[3][4] == 1):
            player1Score += 1
        if (board[1][2] == 1 and board[2][3] == 1 and
               board[3][4] == 1 and board[4][5] == 1):
            player1Score += 1
        if (board[2][3] == 1 and board[3][4] == 1 and
               board[4][5] == 1 and board[5][6] == 1):
            player1Score += 1
        if (board[0][2] == 1 and board[1][3] == 1 and
               board[2][4] == 1 and board[3][5] == 1):
            player1Score += 1
        if (board[1][3] == 1 and board[2][4] == 1 and
               board[3][5] == 1 and board[4][6] == 1):
            player1Score += 1
        if (board[0][3] == 1 and board[1][4] == 1 and
               board[2][5] == 1 and board[3][6] == 1):
            player1Score += 1

        if (board[0][3] == 1 and board[1][2] == 1 and
               board[2][1] == 1 and board[3][0] == 1):
            player1Score += 1
        if (board[0][4] == 1 and board[1][3] == 1 and
               board[2][2] == 1 and board[3][1] == 1):
            player1Score += 1
        if (board[1][3] == 1 and board[2][2] == 1 and
               board[3][1] == 1 and board[4][0] == 1):
            player1Score += 1
        if (board[0][5] == 1 and board[1][4] == 1 and
               board[2][3] == 1 and board[3][2] == 1):
            player1Score += 1
        if (board[1][4] == 1 and board[2][3] == 1 and
               board[3][2] == 1 and board[4][1] == 1):
            player1Score += 1
        if (board[2][3] == 1 and board[3][2] == 1 and
               board[4][1] == 1 and board[5][0] == 1):
            player1Score += 1
        if (board[0][6] == 1 and board[1][5] == 1 and
               board[2][4] == 1 and board[3][3] == 1):
            player1Score += 1
        if (board[1][5] == 1 and board[2][4] == 1 and
               board[3][3] == 1 and board[4][2] == 1):
            player1Score += 1
        if (board[2][4] == 1 and board[3][3] == 1 and
               board[4][2] == 1 and board[5][1] == 1):
            player1Score += 1
        if (board[1][6] == 1 and board[2][5] == 1 and
               board[3][4] == 1 and board[4][3] == 1):
            player1Score += 1
        if (board[2][5] == 1 and board[3][4] == 1 and
               board[4][3] == 1 and board[5][2] == 1):
            player1Score += 1
        if (board[2][6] == 1 and board[3][5] == 1 and
               board[4][4] == 1 and board[5][3] == 1):
            player1Score += 1

        # Check player 2
        if (board[2][0] == 2 and board[3][1] == 2 and
               board[4][2] == 2 and board[5][3] == 2):
            player2Score += 1
        if (board[1][0] == 2 and board[2][1] == 2 and
               board[3][2] == 2 and board[4][3] == 2):
            player2Score += 1
        if (board[2][1] == 2 and board[3][2] == 2 and
               board[4][3] == 2 and board[5][4] == 2):
            player2Score += 1
        if (board[0][0] == 2 and board[1][1] == 2 and
               board[2][2] == 2 and board[3][3] == 2):
            player2Score += 1
        if (board[1][1] == 2 and board[2][2] == 2 and
               board[3][3] == 2 and board[4][4] == 2):
            player2Score += 1
        if (board[2][2] == 2 and board[3][3] == 2 and
               board[4][4] == 2 and board[5][5] == 2):
            player2Score += 1
        if (board[0][1] == 2 and board[1][2] == 2 and
               board[2][3] == 2 and board[3][4] == 2):
            player2Score += 1
        if (board[1][2] == 2 and board[2][3] == 2 and
               board[3][4] == 2 and board[4][5] == 2):
            player2Score += 1
        if (board[2][3] == 2 and board[3][4] == 2 and
               board[4][5] == 2 and board[5][6] == 2):
            player2Score += 1
        if (board[0][2] == 2 and board[1][3] == 2 and
               board[2][4] == 2 and board[3][5] == 2):
            player2Score += 1
        if (board[1][3] == 2 and board[2][4] == 2 and
               board[3][5] == 2 and board[4][6] == 2):
            player2Score += 1
        if (board[0][3] == 2 and board[1][4] == 2 and
               board[2][5] == 2 and board[3][6] == 2):
            player2Score += 1

        if (board[0][3] == 2 and board[1][2] == 2 and
               board[2][1] == 2 and board[3][0] == 2):
            player2Score += 1
        if (board[0][4] == 2 and board[1][3] == 2 and
               board[2][2] == 2 and board[3][1] == 2):
            player2Score += 1
        if (board[1][3] == 2 and board[2][2] == 2 and
               board[3][1] == 2 and board[4][0] == 2):
            player2Score += 1
        if (board[0][5] == 2 and board[1][4] == 2 and
               board[2][3] == 2 and board[3][2] == 2):
            player2Score += 1
        if (board[1][4] == 2 and board[2][3] == 2 and
               board[3][2] == 2 and board[4][1] == 2):
            player2Score += 1
        if (board[2][3] == 2 and board[3][2] == 2 and
               board[4][1] == 2 and board[5][0] == 2):
            player2Score += 1
        if (board[0][6] == 2 and board[1][5] == 2 and
               board[2][4] == 2 and board[3][3] == 2):
            player2Score += 1
        if (board[1][5] == 2 and board[2][4] == 2 and
               board[3][3] == 2 and board[4][2] == 2):
            player2Score += 1
        if (board[2][4] == 2 and board[3][3] == 2 and
               board[4][2] == 2 and board[5][1] == 2):
            player2Score += 1
        if (board[1][6] == 2 and board[2][5] == 2 and
               board[3][4] == 2 and board[4][3] == 2):
            player2Score += 1
        if (board[2][5] == 2 and board[3][4] == 2 and
               board[4][3] == 2 and board[5][2] == 2):
            player2Score += 1
        if (board[2][6] == 2 and board[3][5] == 2 and
               board[4][4] == 2 and board[5][3] == 2):
            player2Score += 1
        
        return player1Score, player2Score
            
    # Calculate the number of 4-in-a-row each player has
    def countScore(self):
        self.player1Score = 0;
        self.player2Score = 0;

        # Check horizontally
        for row in self.gameBoard:
            # Check player 1
            if row[0:4] == [1]*4:
                self.player1Score += 1
            if row[1:5] == [1]*4:
                self.player1Score += 1
            if row[2:6] == [1]*4:
                self.player1Score += 1
            if row[3:7] == [1]*4:
                self.player1Score += 1
            # Check player 2
            if row[0:4] == [2]*4:
                self.player2Score += 1
            if row[1:5] == [2]*4:
                self.player2Score += 1
            if row[2:6] == [2]*4:
                self.player2Score += 1
            if row[3:7] == [2]*4:
                self.player2Score += 1

        # Check vertically
        for j in range(7):
            # Check player 1
            if (self.gameBoard[0][j] == 1 and self.gameBoard[1][j] == 1 and
                   self.gameBoard[2][j] == 1 and self.gameBoard[3][j] == 1):
                self.player1Score += 1
            if (self.gameBoard[1][j] == 1 and self.gameBoard[2][j] == 1 and
                   self.gameBoard[3][j] == 1 and self.gameBoard[4][j] == 1):
                self.player1Score += 1
            if (self.gameBoard[2][j] == 1 and self.gameBoard[3][j] == 1 and
                   self.gameBoard[4][j] == 1 and self.gameBoard[5][j] == 1):
                self.player1Score += 1
            # Check player 2
            if (self.gameBoard[0][j] == 2 and self.gameBoard[1][j] == 2 and
                   self.gameBoard[2][j] == 2 and self.gameBoard[3][j] == 2):
                self.player2Score += 1
            if (self.gameBoard[1][j] == 2 and self.gameBoard[2][j] == 2 and
                   self.gameBoard[3][j] == 2 and self.gameBoard[4][j] == 2):
                self.player2Score += 1
            if (self.gameBoard[2][j] == 2 and self.gameBoard[3][j] == 2 and
                   self.gameBoard[4][j] == 2 and self.gameBoard[5][j] == 2):
                self.player2Score += 1

        # Check diagonally

        # Check player 1
        if (self.gameBoard[2][0] == 1 and self.gameBoard[3][1] == 1 and
               self.gameBoard[4][2] == 1 and self.gameBoard[5][3] == 1):
            self.player1Score += 1
        if (self.gameBoard[1][0] == 1 and self.gameBoard[2][1] == 1 and
               self.gameBoard[3][2] == 1 and self.gameBoard[4][3] == 1):
            self.player1Score += 1
        if (self.gameBoard[2][1] == 1 and self.gameBoard[3][2] == 1 and
               self.gameBoard[4][3] == 1 and self.gameBoard[5][4] == 1):
            self.player1Score += 1
        if (self.gameBoard[0][0] == 1 and self.gameBoard[1][1] == 1 and
               self.gameBoard[2][2] == 1 and self.gameBoard[3][3] == 1):
            self.player1Score += 1
        if (self.gameBoard[1][1] == 1 and self.gameBoard[2][2] == 1 and
               self.gameBoard[3][3] == 1 and self.gameBoard[4][4] == 1):
            self.player1Score += 1
        if (self.gameBoard[2][2] == 1 and self.gameBoard[3][3] == 1 and
               self.gameBoard[4][4] == 1 and self.gameBoard[5][5] == 1):
            self.player1Score += 1
        if (self.gameBoard[0][1] == 1 and self.gameBoard[1][2] == 1 and
               self.gameBoard[2][3] == 1 and self.gameBoard[3][4] == 1):
            self.player1Score += 1
        if (self.gameBoard[1][2] == 1 and self.gameBoard[2][3] == 1 and
               self.gameBoard[3][4] == 1 and self.gameBoard[4][5] == 1):
            self.player1Score += 1
        if (self.gameBoard[2][3] == 1 and self.gameBoard[3][4] == 1 and
               self.gameBoard[4][5] == 1 and self.gameBoard[5][6] == 1):
            self.player1Score += 1
        if (self.gameBoard[0][2] == 1 and self.gameBoard[1][3] == 1 and
               self.gameBoard[2][4] == 1 and self.gameBoard[3][5] == 1):
            self.player1Score += 1
        if (self.gameBoard[1][3] == 1 and self.gameBoard[2][4] == 1 and
               self.gameBoard[3][5] == 1 and self.gameBoard[4][6] == 1):
            self.player1Score += 1
        if (self.gameBoard[0][3] == 1 and self.gameBoard[1][4] == 1 and
               self.gameBoard[2][5] == 1 and self.gameBoard[3][6] == 1):
            self.player1Score += 1

        if (self.gameBoard[0][3] == 1 and self.gameBoard[1][2] == 1 and
               self.gameBoard[2][1] == 1 and self.gameBoard[3][0] == 1):
            self.player1Score += 1
        if (self.gameBoard[0][4] == 1 and self.gameBoard[1][3] == 1 and
               self.gameBoard[2][2] == 1 and self.gameBoard[3][1] == 1):
            self.player1Score += 1
        if (self.gameBoard[1][3] == 1 and self.gameBoard[2][2] == 1 and
               self.gameBoard[3][1] == 1 and self.gameBoard[4][0] == 1):
            self.player1Score += 1
        if (self.gameBoard[0][5] == 1 and self.gameBoard[1][4] == 1 and
               self.gameBoard[2][3] == 1 and self.gameBoard[3][2] == 1):
            self.player1Score += 1
        if (self.gameBoard[1][4] == 1 and self.gameBoard[2][3] == 1 and
               self.gameBoard[3][2] == 1 and self.gameBoard[4][1] == 1):
            self.player1Score += 1
        if (self.gameBoard[2][3] == 1 and self.gameBoard[3][2] == 1 and
               self.gameBoard[4][1] == 1 and self.gameBoard[5][0] == 1):
            self.player1Score += 1
        if (self.gameBoard[0][6] == 1 and self.gameBoard[1][5] == 1 and
               self.gameBoard[2][4] == 1 and self.gameBoard[3][3] == 1):
            self.player1Score += 1
        if (self.gameBoard[1][5] == 1 and self.gameBoard[2][4] == 1 and
               self.gameBoard[3][3] == 1 and self.gameBoard[4][2] == 1):
            self.player1Score += 1
        if (self.gameBoard[2][4] == 1 and self.gameBoard[3][3] == 1 and
               self.gameBoard[4][2] == 1 and self.gameBoard[5][1] == 1):
            self.player1Score += 1
        if (self.gameBoard[1][6] == 1 and self.gameBoard[2][5] == 1 and
               self.gameBoard[3][4] == 1 and self.gameBoard[4][3] == 1):
            self.player1Score += 1
        if (self.gameBoard[2][5] == 1 and self.gameBoard[3][4] == 1 and
               self.gameBoard[4][3] == 1 and self.gameBoard[5][2] == 1):
            self.player1Score += 1
        if (self.gameBoard[2][6] == 1 and self.gameBoard[3][5] == 1 and
               self.gameBoard[4][4] == 1 and self.gameBoard[5][3] == 1):
            self.player1Score += 1

        # Check player 2
        if (self.gameBoard[2][0] == 2 and self.gameBoard[3][1] == 2 and
               self.gameBoard[4][2] == 2 and self.gameBoard[5][3] == 2):
            self.player2Score += 1
        if (self.gameBoard[1][0] == 2 and self.gameBoard[2][1] == 2 and
               self.gameBoard[3][2] == 2 and self.gameBoard[4][3] == 2):
            self.player2Score += 1
        if (self.gameBoard[2][1] == 2 and self.gameBoard[3][2] == 2 and
               self.gameBoard[4][3] == 2 and self.gameBoard[5][4] == 2):
            self.player2Score += 1
        if (self.gameBoard[0][0] == 2 and self.gameBoard[1][1] == 2 and
               self.gameBoard[2][2] == 2 and self.gameBoard[3][3] == 2):
            self.player2Score += 1
        if (self.gameBoard[1][1] == 2 and self.gameBoard[2][2] == 2 and
               self.gameBoard[3][3] == 2 and self.gameBoard[4][4] == 2):
            self.player2Score += 1
        if (self.gameBoard[2][2] == 2 and self.gameBoard[3][3] == 2 and
               self.gameBoard[4][4] == 2 and self.gameBoard[5][5] == 2):
            self.player2Score += 1
        if (self.gameBoard[0][1] == 2 and self.gameBoard[1][2] == 2 and
               self.gameBoard[2][3] == 2 and self.gameBoard[3][4] == 2):
            self.player2Score += 1
        if (self.gameBoard[1][2] == 2 and self.gameBoard[2][3] == 2 and
               self.gameBoard[3][4] == 2 and self.gameBoard[4][5] == 2):
            self.player2Score += 1
        if (self.gameBoard[2][3] == 2 and self.gameBoard[3][4] == 2 and
               self.gameBoard[4][5] == 2 and self.gameBoard[5][6] == 2):
            self.player2Score += 1
        if (self.gameBoard[0][2] == 2 and self.gameBoard[1][3] == 2 and
               self.gameBoard[2][4] == 2 and self.gameBoard[3][5] == 2):
            self.player2Score += 1
        if (self.gameBoard[1][3] == 2 and self.gameBoard[2][4] == 2 and
               self.gameBoard[3][5] == 2 and self.gameBoard[4][6] == 2):
            self.player2Score += 1
        if (self.gameBoard[0][3] == 2 and self.gameBoard[1][4] == 2 and
               self.gameBoard[2][5] == 2 and self.gameBoard[3][6] == 2):
            self.player2Score += 1

        if (self.gameBoard[0][3] == 2 and self.gameBoard[1][2] == 2 and
               self.gameBoard[2][1] == 2 and self.gameBoard[3][0] == 2):
            self.player2Score += 1
        if (self.gameBoard[0][4] == 2 and self.gameBoard[1][3] == 2 and
               self.gameBoard[2][2] == 2 and self.gameBoard[3][1] == 2):
            self.player2Score += 1
        if (self.gameBoard[1][3] == 2 and self.gameBoard[2][2] == 2 and
               self.gameBoard[3][1] == 2 and self.gameBoard[4][0] == 2):
            self.player2Score += 1
        if (self.gameBoard[0][5] == 2 and self.gameBoard[1][4] == 2 and
               self.gameBoard[2][3] == 2 and self.gameBoard[3][2] == 2):
            self.player2Score += 1
        if (self.gameBoard[1][4] == 2 and self.gameBoard[2][3] == 2 and
               self.gameBoard[3][2] == 2 and self.gameBoard[4][1] == 2):
            self.player2Score += 1
        if (self.gameBoard[2][3] == 2 and self.gameBoard[3][2] == 2 and
               self.gameBoard[4][1] == 2 and self.gameBoard[5][0] == 2):
            self.player2Score += 1
        if (self.gameBoard[0][6] == 2 and self.gameBoard[1][5] == 2 and
               self.gameBoard[2][4] == 2 and self.gameBoard[3][3] == 2):
            self.player2Score += 1
        if (self.gameBoard[1][5] == 2 and self.gameBoard[2][4] == 2 and
               self.gameBoard[3][3] == 2 and self.gameBoard[4][2] == 2):
            self.player2Score += 1
        if (self.gameBoard[2][4] == 2 and self.gameBoard[3][3] == 2 and
               self.gameBoard[4][2] == 2 and self.gameBoard[5][1] == 2):
            self.player2Score += 1
        if (self.gameBoard[1][6] == 2 and self.gameBoard[2][5] == 2 and
               self.gameBoard[3][4] == 2 and self.gameBoard[4][3] == 2):
            self.player2Score += 1
        if (self.gameBoard[2][5] == 2 and self.gameBoard[3][4] == 2 and
               self.gameBoard[4][3] == 2 and self.gameBoard[5][2] == 2):
            self.player2Score += 1
        if (self.gameBoard[2][6] == 2 and self.gameBoard[3][5] == 2 and
               self.gameBoard[4][4] == 2 and self.gameBoard[5][3] == 2):
            self.player2Score += 1

########################## CODE WAS REMOVED FROM HERE AND PASTED IN LAST CELL################################### 
    def maskify1(self, arr, mask):
        arr1 = [row[:] for row in arr] #arr.copy()
        if len(arr) != len(mask) or len(arr[1]) != len(mask[1]):
            print("Error: Size of array passed and mask is not same!")
            return []
        #print("Earlier:", arr1)
        for i in range(len(arr1)):
            for j in range(len(arr1[1])):
                arr1[i][j] = arr[i][j] * mask[i][j]
        #print("Later:", arr1)
        #print(sum(sum(arr1,[])))
        return sum(sum(arr1,[]))

    def maskify(self, arr, mask):
        h = [[0,0,0,0,0,0,0],[1,1,1,1,1,1,1],[2,2,2,2,2,2,2],[3,3,3,3,3,3,3],[4,4,4,4,4,4,4],[5,5,5,5,5,5,5]]
        arr1 = [row[:] for row in arr] #arr.copy()
        if len(arr) != len(mask) or len(arr[1]) != len(mask[1]):
            print("Error: Size of array passed and mask is not same!")
            return []
        #print("Earlier:", arr1)
        for i in range(len(arr1)):
            for j in range(len(arr1[1])):
                if mask[i][j] == "$":
                    arr[i][j] = -abs(arr[i][j]) ** h[i][j]
                elif mask[i][j] == "#":
                    arr[i][j] = abs(arr[i][j]) ** h[i][j]
                else:
                    #if i == len(arr1):
                    #    arr[i][j] = arr[i][j] * mask[i][j] * h[i][j]
                    #else:
                    #    arr[i][j] = arr[i][j] * mask[i][j] * h[i][j]
                    arr1[i][j] = (arr[i][j] * mask[i][j]) ** (h[i][j])
        #print("Later:", arr1)
        #print(sum(sum(arr1,[])))
        return sum(sum(arr1,[]))
    
    def transpose(self, l1):
        l2 =[[row[i] for row in l1] for i in range(len(l1[0]))]
        return l2
    
    def sum_2d(self, arr):
        try:
            return sum(sum(arr,[]))
        except:
            print("Error: The passed array might not be a 2D array.")
            return sum(sum(arr,[]))
    
    def gen_masks3(self):
        mask = []
        
        mask.append([[0,0,0,0,0,0,0],[1,1,1,1,1,1,1],[2,2,2,2,2,2,2],[3,3,3,3,3,3,3],[4,4,4,4,4,4,4],[5,5,5,5,5,5,5]])
        
        #len_curr = len(mask)
        #print("Length of mask as of now:", len_curr)
        ## Some additional code goes here ##
        #for i in range(len_curr):
        #    temp = self.transpose(mask[i])
        #    mask.append(temp)
        #print("Now the length of mask is", len(mask))
        return mask

    def gen_masks(self):
        mask = []
        #Diogonal from top left to bottom right
        mask.append([[1,"$","$","$",0,0,0],["#",1,"$","$",0,0,0],["#","#",1,"$",0,0,0],["#","#","#",1,0,0,0],["#","#","#","#",0,0,0],[0,0,0,0,0,0,0]])
        mask.append([["$","$","$","$",0,0,0],[1,"$","$","$",0,0,0],["#",1,"$","$",0,0,0],["#","#",1,"$",0,0,0],["#","#","#",1,0,0,0],["#","#","#","#",0,0,0]])
        mask.append([[0,0,0,0,0,0,0],["$","$","$","$",0,0,0],[1,"$","$","$",0,0,0],["#",1,"$","$",0,0,0],["#","#",1,"$",0,0,0],["#","#","#",1,0,0,0]])
        mask.append([[0,1,"$","$","$",0,0],[0,"#",1,"$","$",0,0],[0,"#","#",1,"$",0,0],[0,"#","#","#",1,0,0],[0,"#","#","#","#",0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,"$","$","$","$",0,0],[0,1,"$","$","$",0,0],[0,"#",1,"$","$",0,0],[0,"#","#",1,"$",0,0],[0,"#","#","#",1,0,0],[0,"#","#","#","#",0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,"$","$","$","$",0,0],[0,1,"$","$","$",0,0],[0,"#",1,"$","$",0,0],[0,"#","#",1,"$",0,0],[0,"#","#","#",1,0,0]])
        mask.append([[0,0,1,"$","$","$",0],[0,0,"#",1,"$","$",0],[0,0,"#","#",1,"$",0],[0,0,"#","#","#",1,0],[0,0,"#","#","#","#",0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,"$","$","$","$",0],[0,0,1,"$","$","$",0],[0,0,"#",1,"$","$",0],[0,0,"#","#",1,"$",0],[0,0,"#","#","#",1,0],[0,0,"#","#","#","#",0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,"$","$","$","$",0],[0,0,1,"$","$","$",0],[0,0,"#",1,"$","$",0],[0,0,"#","#",1,"$",0],[0,0,"#","#","#",1,0]])
        mask.append([[0,0,0,1,"$","$","$"],[0,0,0,"#",1,"$","$"],[0,0,0,"#","#",1,"$"],[0,0,0,"#","#","#",1],[0,0,0,"#","#","#","#"],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,"$","$","$","$"],[0,0,0,1,"$","$","$"],[0,0,0,"#",1,"$","$"],[0,0,0,"#","#",1,"$"],[0,0,0,"#","#","#",1],[0,0,0,"#","#","#","#"]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,"$","$","$","$"],[0,0,0,1,"$","$","$"],[0,0,0,"#",1,"$","$"],[0,0,0,"#","#",1,"$"],[0,0,0,"#","#","#",1]])
        #Diogonal from bottom left to top right
        mask.append([["$","$","$",1,0,0,0],["$","$",1,"#",0,0,0],["$",1,"#","#",0,0,0],[1,"#","#","#",0,0,0],["#","#","#","#",0,0,0],[0,0,0,0,0,0,0]])
        mask.append([["$","$","$","$",0,0,0],["$","$","$",1,0,0,0],["$","$",1,"#",0,0,0],["$",1,"#","#",0,0,0],[1,"#","#","#",0,0,0],["#","#","#","#",0,0,0]])
        mask.append([[0,0,0,0,0,0,0],["$","$","$","$",0,0,0],["$","$","$",1,0,0,0],["$","$",1,"#",0,0,0],["$",1,"#","#",0,0,0],[1,"#","#","#",0,0,0]])
        mask.append([[0,"$","$","$",1,0,0],[0,"$","$",1,"#",0,0],[0,"$",1,"#","#",0,0],[0,1,"#","#","#",0,0],[0,"#","#","#","#",0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,"$","$","$","$",0,0],[0,"$","$","$",1,0,0],[0,"$","$",1,"#",0,0],[0,"$",1,"#","#",0,0],[0,1,"#","#","#",0,0],[0,"#","#","#","#",0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,"$","$","$","$",0,0],[0,"$","$","$",1,0,0],[0,"$","$",1,"#",0,0],[0,"$",1,"#","#",0,0],[0,1,"#","#","#",0,0]])
        mask.append([[0,0,"$","$","$",1,0],[0,0,"$","$",1,"#",0],[0,0,"$",1,"#","#",0],[0,0,1,"#","#","#",0],[0,0,"#","#","#","#",0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,"$","$","$","$",0],[0,0,"$","$","$",1,0],[0,0,"$","$",1,"#",0],[0,0,"$",1,"#","#",0],[0,0,1,"#","#","#",0],[0,0,"#","#","#","#",0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,"$","$","$","$",0],[0,0,"$","$","$",1,0],[0,0,"$","$",1,"#",0],[0,0,"$",1,"#","#",0],[0,0,1,"#","#","#",0]])
        mask.append([[0,0,0,"$","$","$",1],[0,0,0,"$","$",1,"#"],[0,0,0,"$",1,"#","#"],[0,0,0,1,"#","#","#"],[0,0,0,"#","#","#","#"],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,"$","$","$","$"],[0,0,0,"$","$","$",1],[0,0,0,"$","$",1,"#"],[0,0,0,"$",1,"#","#"],[0,0,0,1,"#","#","#"],[0,0,0,"#","#","#","#"]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,"$","$","$","$"],[0,0,0,"$","$","$",1],[0,0,0,"$","$",1,"#"],[0,0,0,"$",1,"#","#"],[0,0,0,1,"#","#","#"]])
        #Horizontal check
        mask.append([[1,1,1,1,0,0,0],["#","#","#","#",0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([["$","$","$","$",0,0,0],[1,1,1,1,0,0,0],["#","#","#","#",0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],["$","$","$","$",0,0,0],[1,1,1,1,0,0,0],["#","#","#","#",0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],["$","$","$","$",0,0,0],[1,1,1,1,0,0,0],["#","#","#","#",0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],["$","$","$","$",0,0,0],[1,1,1,1,0,0,0],["#","#","#","#",0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],["$","$","$","$",0,0,0],[1,1,1,1,0,0,0]])
        mask.append([[0,1,1,1,1,0,0],[0,"#","#","#","#",0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,"$","$","$","$",0,0],[0,1,1,1,1,0,0],[0,"#","#","#","#",0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,"$","$","$","$",0,0],[0,1,1,1,1,0,0],[0,"#","#","#","#",0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,"$","$","$","$",0,0],[0,1,1,1,1,0,0],[0,"#","#","#","#",0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,"$","$","$","$",0,0],[0,1,1,1,1,0,0],[0,"#","#","#","#",0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,"$","$","$","$",0,0],[0,1,1,1,1,0,0]])
        mask.append([[0,0,1,1,1,1,0],[0,0,"#","#","#","#",0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,"$","$","$","$",0],[0,0,1,1,1,1,0],[0,0,"#","#","#","#",0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,"$","$","$","$",0],[0,0,1,1,1,1,0],[0,0,"#","#","#","#",0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,"$","$","$","$",0],[0,0,1,1,1,1,0],[0,0,"#","#","#","#",0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,"$","$","$","$",0],[0,0,1,1,1,1,0],[0,0,"#","#","#","#",0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,"$","$","$","$",0],[0,0,1,1,1,1,0]])
        mask.append([[0,0,0,1,1,1,1],[0,0,0,"#","#","#","#"],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,"$","$","$","$"],[0,0,0,1,1,1,1],[0,0,0,"#","#","#","#"],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,"$","$","$","$"],[0,0,0,1,1,1,1],[0,0,0,"#","#","#","#"],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,"$","$","$","$"],[0,0,0,1,1,1,1],[0,0,0,"#","#","#","#"],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,"$","$","$","$"],[0,0,0,1,1,1,1],[0,0,0,"#","#","#","#"]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,"$","$","$","$"],[0,0,0,1,1,1,1]])
        
        #len_curr = len(mask)
        #print("Length of mask as of now:", len_curr)
        ## Some additional code goes here ##
        #for i in range(len_curr):
        #    temp = self.transpose(mask[i])
        #    mask.append(temp)
        #print("Now the length of mask is", len(mask))
        return mask

    def eval_func2(self, board):
        ct1 = self.currentTurn
        board_copy = [row[:] for row in board] #board.copy()
        #print(board_copy)
        #print("Before")
        #self.printGameBoard_pass(board_copy)
        return_eval = 0
        if ct1 == 1:
            ct2 = 2
            score1, score2 = self.countScore_custom(board_copy)
        else:
            ct2 = 1
            score2, score1 = self.countScore_custom(board_copy)
        
        return_eval = score1 - score2
        return return_eval

    def eval_func(self, board):
        mask_of = self.currentTurn
        board_copy1 = [row[:] for row in board] #board.copy()
        board_copy2 = [row[:] for row in board] #board.copy()
        
        #print(board_copy)
        #print("Before")
        #self.printGameBoard_pass(board_copy)
        
        for i in range(6):
            for j in range(7):
                if board_copy1[i][j] == mask_of:
                    board_copy1[i][j] = 1
                elif board_copy1[i][j] == 0:
                    board_copy1[i][j] = 0
                else:
                    board_copy1[i][j] = -1
        
        #for i in range(6):
        #    for j in range(7):
        #        if board_copy2[i][j] != mask_of:
        #            board_copy2[i][j] = 1
        #        elif board_copy2[i][j] == 0:
        #            board_copy2[i][j] = 0
        #        else:
        #            board_copy2[i][j] = -1
        
        #self.printGameBoard()
        masks = self.gen_masks()
        
        return_eval = 0
        for mask in masks:
            #self.printGameBoard_pass(board)
            #self.printGameBoard_pass(mask)
            val1 = self.maskify(board_copy1, mask)
            #val2 = self.maskify(board_copy2, mask)
            #print("Score:", val1)
            #if val1 >= 3:
            #    return_eval += 1
            #else:
            #    return_eval -= 1
            return_eval += (val1) + self.eval_func2(board_copy2)
        #print("After")
        #self.printGameBoard_pass(board_copy)
        #print("Eval score of above board", return_eval)
        return return_eval
        
    def eval_func1(self, board):
        p1 = self.currentTurn
        board_copy = [row[:] for row in board] #board.copy()
        if p1 == 1:
            p2 = 2
        else:
            p2 = 1
        
        

In [385]:
#!/usr/bin/env python

# Written by Chris Conly based on C++
# code provided by Dr. Vassilis Athitsos
# Written to be Python 2.4 compatible for omega

import sys
#from MaxConnect4Game import *

def oneMoveGame(currentGame):
    if currentGame.pieceCount == 42:    # Is the board full already?
        print('BOARD FULL\n\nGame Over!\n')
        sys.exit(0)

    currentGame.aiPlay() # Make a move (only random is implemented)

    print('Game state after move:')
    currentGame.printGameBoard()

    currentGame.countScore()
    print('Score: Player 1 = %d, Player 2 = %d\n' % (currentGame.player1Score, currentGame.player2Score))

    currentGame.printGameBoardToFile()
    currentGame.gameFile.close()


def interactiveGame(currentGame):
    player = input("Enter your player name: ")
    
    # Fill me in
    if currentGame.pieceCount == 42:    # Is the board full already?
        print('BOARD FULL\n\nGame Over!\n')
        sys.exit(0)

    ret = -1
    col = 1
    while currentGame.pieceCount != 42:
        print("\nPlayer: AI")
        currentGame.aiPlay() # Make a move (only random is implemented)
        
        print('Game state after move:')
        currentGame.printGameBoard()
        currentGame.countScore()
        print('Score: Player 1 = %d, Player 2 = %d\n' % (currentGame.player1Score, currentGame.player2Score))
        
        ret = -1
        print("\nPlayer:", player)
        print('Your turn. Enter the column in which you want to enter piece.')
        while ret == -1:
            col = int(input("Enter value from 1 to 7 (0 to stop game): "))
            if col == 0:
                break
            col -= 1
            ret = currentGame.playPiece(col)
            if ret == -1:
                print("Error: Invalid column number. Please enter a value in given range.")
        if col == -1:
            print("User entered zero to stop game.")
            break
        #####currentGame.currentTurn
        if currentGame.currentTurn == 1:
            currentGame.currentTurn = 2
        elif currentGame.currentTurn == 2:
            currentGame.currentTurn = 1
        
        print('Game state after your move:')
        currentGame.printGameBoard()
        currentGame.countScore()
        print('Score: Player 1 = %d, Player 2 = %d\n' % (currentGame.player1Score, currentGame.player2Score))
        
    #currentGame.printGameBoardToFile()
    #currentGame.gameFile.close()
    
    print('Final score: Player 1 = %d, Player 2 = %d\n' % (currentGame.player1Score, currentGame.player2Score))
    #sys.exit('Interactive mode ended')


def main(argv):
    # Make sure we have enough command-line arguments
    if len(argv) != 5:
        print('Four command-line arguments are needed:')
        print('Usage: %s interactive [input_file] [computer-next/human-next] [depth]' % argv[0])
        print('or: %s one-move [input_file] [output_file] [depth]' % argv[0])
        sys.exit(2)

    game_mode, inFile = argv[1:3]

    if not game_mode == 'interactive' and not game_mode == 'one-move':
        print('%s is an unrecognized game mode' % game_mode)
        sys.exit(2)

    currentGame = maxConnect4Game() # Create a game

    # Try to open the input file
    try:
        currentGame.gameFile = open(inFile, 'r')
    except IOError:
        sys.exit("\nError opening input file.\nCheck file name.\n")

    # Read the initial game state from the file and save in a 2D list
    file_lines = currentGame.gameFile.readlines()
    currentGame.gameBoard = [[int(char) for char in line[0:7]] for line in file_lines[0:-1]]
    currentGame.currentTurn = int(file_lines[-1][0])
    currentGame.gameFile.close()

    print('\nMaxConnect-4 game\n')
    print('Game state before move:')
    #currentGame.gameBoard = [[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]]
    currentGame.printGameBoard()

    # Update a few game variables based on initial state and print the score
    currentGame.checkPieceCount()
    currentGame.countScore()
    print('Score: Player 1 = %d, Player 2 = %d\n' % (currentGame.player1Score, currentGame.player2Score))

    if game_mode == 'interactive':
        interactiveGame(currentGame) # Be sure to pass whatever else you need from the command line
    else: # game_mode == 'one-move'
        # Set up the output file
        outFile = argv[3]
        try:
            currentGame.gameFile = open(outFile, 'w')
        except:
            sys.exit('Error opening output file.')
        oneMoveGame(currentGame) # Be sure to pass any other arguments from the command line you might need.


if __name__ == '__main__':
    main(["o", "interactive", "input2.txt", "output1.txt", "10"])


MaxConnect-4 game

Game state before move:
 -----------------
 |0000000| 
 |0000000| 
 |0200000| 
 |0100002| 
 |0200111| 
 |1100222| 
 -----------------
Score: Player 1 = 0, Player 2 = 0

Enter your player name: p

Player: AI
AI is thinking...
Node: <__main__.node object at 0x00000199D282AD00>
Printing children None
Pred: <__main__.node object at 0x00000199D282AA00>
Node: <__main__.node object at 0x00000199D282AFD0>
Printing children None
Pred: <__main__.node object at 0x00000199D282AEB0>
Node: <__main__.node object at 0x00000199D282AA90>
Printing children None
Pred: <__main__.node object at 0x00000199D282A3A0>
Node: <__main__.node object at 0x00000199D282A3A0>
Printing children [<__main__.node object at 0x00000199D282AAF0>, <__main__.node object at 0x00000199D282A9A0>, <__main__.node object at 0x00000199D282AA90>]
Pred: <__main__.node object at 0x00000199D282ADC0>
Root: <__main__.node object at 0x00000199D282ADC0>
Printing children [<__main__.node object at 0x00000199D282AA00>, <__ma

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [185]:
import random
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

depth = 10
branch_factor = 7
#print(random.randint(0,3))
a = []
for i in range(depth):
    a.append([])
print(a)

level = []
for i in range(depth - 1):
    level.append(1)
#level = [1,1,1,1,1,1,1,1,1]
index1 = len(level) - 1
index2 = len(a) - 1
while sum(level) < 63:
    # play 10 moves depending upon level matrix
    # if set of moves not possible then write score as -1
    game_eval = random.randint(0,5)
    a[index2].append(game_eval)
    level[index1] += 1
    #print(a)
    
    #check if level matrix is valid
    for i in range(len(level)-1, 0, -1):
        if level[i] > branch_factor:
            level[i] = 1
            level[i-1] += 1
    if level[0] > branch_factor:
        break
    
#print(level)
#print(a)
print(len(a[len(a)-1]))
    
max_flag = depth % 2
temp = []
c = 0
#for i in range(len(a[len(a)-1])):
#    if c == 0 or c%branch_factor != 0:
#        temp.append(a[len(a)-1][i])
#    else:
#        if max_flag == 1 and len(temp) == branch_factor:
#            a[len(a)-2].append(max(temp))
#        elif max_flag == 0 and len(temp) == branch_factor:
#            a[len(a)-2].append(min(temp))
#        temp = []
#        temp.append(a[len(a)-1][i])
#    c += 1



for k in range(1,len(level)+1):
#for k in range(1,3):
    max_flag += 1
    max_flag = max_flag % 2
    #print("NEW!!")
    #input()
    for i in range(len(a[len(a)-k])):
        if c == 0 or c%branch_factor != 0:
            temp.append(a[len(a)-k][i])
        else:
            if max_flag == 1 and len(temp) == branch_factor:
                a[len(a)-k-1].append(max(temp))
                #print(temp)
                #print(max(temp))
            elif max_flag == 0 and len(temp) == branch_factor:
                a[len(a)-k-1].append(min(temp))
                #print(temp)
                #print(max(temp))
            temp = []
            temp.append(a[len(a)-k][i])
        c += 1
    if max_flag == 1 and len(temp) == branch_factor:
        a[len(a)-k-1].append(max(temp))
        #print(temp)
        #print(max(temp))
    elif max_flag == 0 and len(temp) == branch_factor:
        a[len(a)-k-1].append(min(temp))
        #print(temp)
        #print(max(temp))
    c = 0
    temp = []
        
print(len(a[len(a)-3]))
print(len(a[len(a)-2]))


[[], [], [], [], [], [], [], [], [], []]
40353606


KeyboardInterrupt: 

In [179]:
for i in range(depth):
    print(a[i])

[3]
[5, 3, 3]
[2, 5, 4, 3, 3, 3, 2, 2, 3]
[3, 5, 2, 5, 5, 5, 5, 5, 4, 3, 3, 5, 5, 5, 3, 5, 3, 3, 5, 2, 5, 2, 4, 5, 5, 3, 4]
[1, 1, 3, 5, 4, 1, 2, 2, 0, 5, 0, 0, 5, 4, 2, 3, 3, 5, 5, 4, 1, 5, 4, 5, 4, 0, 2, 3, 2, 2, 3, 0, 1, 4, 5, 3, 2, 5, 4, 0, 1, 5, 1, 3, 1, 5, 1, 4, 2, 3, 0, 0, 0, 3, 1, 5, 4, 2, 2, 1, 3, 4, 5, 2, 2, 2, 4, 2, 3, 5, 0, 4, 3, 1, 5, 1, 1, 3, 0, 4, 4]


In [180]:
print(a[4])
print(len(a[len(a)-1]))

[1, 1, 3, 5, 4, 1, 2, 2, 0, 5, 0, 0, 5, 4, 2, 3, 3, 5, 5, 4, 1, 5, 4, 5, 4, 0, 2, 3, 2, 2, 3, 0, 1, 4, 5, 3, 2, 5, 4, 0, 1, 5, 1, 3, 1, 5, 1, 4, 2, 3, 0, 0, 0, 3, 1, 5, 4, 2, 2, 1, 3, 4, 5, 2, 2, 2, 4, 2, 3, 5, 0, 4, 3, 1, 5, 1, 1, 3, 0, 4, 4]
81


In [181]:
print(a[3])
len(a[3])

[3, 5, 2, 5, 5, 5, 5, 5, 4, 3, 3, 5, 5, 5, 3, 5, 3, 3, 5, 2, 5, 2, 4, 5, 5, 3, 4]


27

In [182]:
print(a[2])
len(a[2])

[2, 5, 4, 3, 3, 3, 2, 2, 3]


9

In [183]:
#print(a[6])
#print(len(a[6]))
#print(len(a[5]))
print(len(a[4]))
print(len(a[3]))
print(len(a[2]))
print(len(a[1]))



81
27
9
3


In [ ]:
    def maskify(self, arr, mask):
        arr1 = [row[:] for row in arr] #arr.copy()
        if len(arr) != len(mask) or len(arr[1]) != len(mask[1]):
            print("Error: Size of array passed and mask is not same!")
            return []
        #print("Earlier:", arr1)
        for i in range(len(arr1)):
            for j in range(len(arr1[1])):
                arr1[i][j] = arr[i][j] * mask[i][j]
        #print("Later:", arr1)
        #print(sum(sum(arr1,[])))
        return sum(sum(arr1,[]))
    
    def transpose(self, l1):
        l2 =[[row[i] for row in l1] for i in range(len(l1[0]))]
        return l2
    
    def sum_2d(self, arr):
        try:
            return sum(sum(arr,[]))
        except:
            print("Error: The passed array might not be a 2D array.")
            return sum(sum(arr,[]))
    
    def gen_masks1(self):
        mask = []
        # Old mask filters
        mask.append([[1,1,1,1,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,1,1,1,1,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,1,1,1,1,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,1,1,1,1],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[1,1,1,1,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,1,1,1,1,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,1,1,1,1,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,1,1,1,1],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[1,1,1,1,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,1,1,1,1,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,1,1,1,1,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,1,1,1,1],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[1,1,1,1,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,1,1,1,1,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,1,1,1,1,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,1,1,1,1],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[1,1,1,1,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,1,1,1,1,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,1,1,1,1,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,1,1,1,1],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[1,1,1,1,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,1,1,1,1,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,1,1,1,1,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,1,1,1,1]])
        #mask.append([[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0]])
        #mask.append([[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0]])
        #mask.append([[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0]])
        #mask.append([[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0]])
        #mask.append([[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0]])
        #mask.append([[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0]])
        #mask.append([[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1]])
        #mask.append([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])
        #mask.append([[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,1,0,0]])
        #mask.append([[0,0,1,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,1,0]])
        #mask.append([[0,0,0,1,0,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
        #mask.append([[0,0,0,1,0,0,0],[0,0,1,0,0,0,0],[0,1,0,0,0,0,0],[1,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,1,0,0,0],[0,0,1,0,0,0,0],[0,1,0,0,0,0,0],[1,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,1,0,0,0],[0,0,1,0,0,0,0],[0,1,0,0,0,0,0],[1,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,1,0,0],[0,0,0,1,0,0,0],[0,0,1,0,0,0,0],[0,1,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,1,0,0],[0,0,0,1,0,0,0],[0,0,1,0,0,0,0],[0,1,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,1,0,0],[0,0,0,1,0,0,0],[0,0,1,0,0,0,0],[0,1,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,1,0],[0,0,0,0,1,0,0],[0,0,0,1,0,0,0],[0,0,1,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,1,0],[0,0,0,0,1,0,0],[0,0,0,1,0,0,0],[0,0,1,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,1,0],[0,0,0,0,1,0,0],[0,0,0,1,0,0,0],[0,0,1,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,1],[0,0,0,0,0,1,0],[0,0,0,0,1,0,0],[0,0,0,1,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,1],[0,0,0,0,0,1,0],[0,0,0,0,1,0,0],[0,0,0,1,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,1],[0,0,0,0,0,1,0],[0,0,0,0,1,0,0],[0,0,0,1,0,0,0]])
        
        # new mask filters
        #mask.append([[1,1,1,1,0,0,0],[1,1,1,1,0,0,0],[1,1,1,1,0,0,0],[1,1,1,1,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[1,1,1,1,0,0,0],[1,1,1,1,0,0,0],[1,1,1,1,0,0,0],[1,1,1,1,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[1,1,1,1,0,0,0],[1,1,1,1,0,0,0],[1,1,1,1,0,0,0],[1,1,1,1,0,0,0]])
        #mask.append([[0,1,1,1,1,0,0],[0,1,1,1,1,0,0],[0,1,1,1,1,0,0],[0,1,1,1,1,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,1,1,1,1,0,0],[0,1,1,1,1,0,0],[0,1,1,1,1,0,0],[0,1,1,1,1,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,1,1,1,1,0,0],[0,1,1,1,1,0,0],[0,1,1,1,1,0,0],[0,1,1,1,1,0,0]])
        #mask.append([[0,0,1,1,1,1,0],[0,0,1,1,1,1,0],[0,0,1,1,1,1,0],[0,0,1,1,1,1,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,1,1,1,1,0],[0,0,1,1,1,1,0],[0,0,1,1,1,1,0],[0,0,1,1,1,1,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,1,1,1,1,0],[0,0,1,1,1,1,0],[0,0,1,1,1,1,0],[0,0,1,1,1,1,0]])
        #mask.append([[0,0,0,1,1,1,1],[0,0,0,1,1,1,1],[0,0,0,1,1,1,1],[0,0,0,1,1,1,1],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,1,1,1,1],[0,0,0,1,1,1,1],[0,0,0,1,1,1,1],[0,0,0,1,1,1,1],[0,0,0,0,0,0,0]])
        #mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,1,1,1,1],[0,0,0,1,1,1,1],[0,0,0,1,1,1,1],[0,0,0,1,1,1,1]])
        
        len_curr = len(mask)
        #print("Length of mask as of now:", len_curr)
        ## Some additional code goes here ##
        #for i in range(len_curr):
        #    temp = self.transpose(mask[i])
        #    mask.append(temp)
        #print("Now the length of mask is", len(mask))
        return mask

    def gen_masks(self):
        mask = []
        #Diogonal from top left to bottom right
        mask.append([[1,"$","$","$",0,0,0],["#",1,"$","$",0,0,0],["#","#",1,"$",0,0,0],["#","#","#",1,0,0,0],["#","#","#","#",0,0,0],[0,0,0,0,0,0,0]])
        mask.append([["$","$","$","$",0,0,0],[1,"$","$","$",0,0,0],["#",1,"$","$",0,0,0],["#","#",1,"$",0,0,0],["#","#","#",1,0,0,0],["#","#","#","#",0,0,0]])
        mask.append([[0,0,0,0,0,0,0],["$","$","$","$",0,0,0],[1,"$","$","$",0,0,0],["#",1,"$","$",0,0,0],["#","#",1,"$",0,0,0],["#","#","#",1,0,0,0]])
        mask.append([[0,1,"$","$","$",0,0],[0,"#",1,"$","$",0,0],[0,"#","#",1,"$",0,0],[0,"#","#","#",1,0,0],[0,"#","#","#","#",0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,"$","$","$","$",0,0],[0,1,"$","$","$",0,0],[0,"#",1,"$","$",0,0],[0,"#","#",1,"$",0,0],[0,"#","#","#",1,0,0],[0,"#","#","#","#",0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,"$","$","$","$",0,0],[0,1,"$","$","$",0,0],[0,"#",1,"$","$",0,0],[0,"#","#",1,"$",0,0],[0,"#","#","#",1,0,0]])
        mask.append([[0,0,1,"$","$","$",0],[0,0,"#",1,"$","$",0],[0,0,"#","#",1,"$",0],[0,0,"#","#","#",1,0],[0,0,"#","#","#","#",0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,"$","$","$","$",0],[0,0,1,"$","$","$",0],[0,0,"#",1,"$","$",0],[0,0,"#","#",1,"$",0],[0,0,"#","#","#",1,0],[0,0,"#","#","#","#",0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,"$","$","$","$",0],[0,0,1,"$","$","$",0],[0,0,"#",1,"$","$",0],[0,0,"#","#",1,"$",0],[0,0,"#","#","#",1,0]])
        mask.append([[0,0,0,1,"$","$","$"],[0,0,0,"#",1,"$","$"],[0,0,0,"#","#",1,"$"],[0,0,0,"#","#","#",1],[0,0,0,"#","#","#","#"],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,"$","$","$","$"],[0,0,0,1,"$","$","$"],[0,0,0,"#",1,"$","$"],[0,0,0,"#","#",1,"$"],[0,0,0,"#","#","#",1],[0,0,0,"#","#","#","#"]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,"$","$","$","$"],[0,0,0,1,"$","$","$"],[0,0,0,"#",1,"$","$"],[0,0,0,"#","#",1,"$"],[0,0,0,"#","#","#",1]])
        #Diogonal from bottom left to top right
        mask.append([["$","$","$",1,0,0,0],["$","$",1,"#",0,0,0],["$",1,"#","#",0,0,0],[1,"#","#","#",0,0,0],["#","#","#","#",0,0,0],[0,0,0,0,0,0,0]])
        mask.append([["$","$","$","$",0,0,0],["$","$","$",1,0,0,0],["$","$",1,"#",0,0,0],["$",1,"#","#",0,0,0],[1,"#","#","#",0,0,0],["#","#","#","#",0,0,0]])
        mask.append([[0,0,0,0,0,0,0],["$","$","$","$",0,0,0],["$","$","$",1,0,0,0],["$","$",1,"#",0,0,0],["$",1,"#","#",0,0,0],[1,"#","#","#",0,0,0]])
        mask.append([[0,"$","$","$",1,0,0],[0,"$","$",1,"#",0,0],[0,"$",1,"#","#",0,0],[0,1,"#","#","#",0,0],[0,"#","#","#","#",0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,"$","$","$","$",0,0],[0,"$","$","$",1,0,0],[0,"$","$",1,"#",0,0],[0,"$",1,"#","#",0,0],[0,1,"#","#","#",0,0],[0,"#","#","#","#",0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,"$","$","$","$",0,0],[0,"$","$","$",1,0,0],[0,"$","$",1,"#",0,0],[0,"$",1,"#","#",0,0],[0,1,"#","#","#",0,0]])
        mask.append([[0,0,"$","$","$",1,0],[0,0,"$","$",1,"#",0],[0,0,"$",1,"#","#",0],[0,0,1,"#","#","#",0],[0,0,"#","#","#","#",0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,"$","$","$","$",0],[0,0,"$","$","$",1,0],[0,0,"$","$",1,"#",0],[0,0,"$",1,"#","#",0],[0,0,1,"#","#","#",0],[0,0,"#","#","#","#",0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,"$","$","$","$",0],[0,0,"$","$","$",1,0],[0,0,"$","$",1,"#",0],[0,0,"$",1,"#","#",0],[0,0,1,"#","#","#",0]])
        mask.append([[0,0,0,"$","$","$",1],[0,0,0,"$","$",1,"#"],[0,0,0,"$",1,"#","#"],[0,0,0,1,"#","#","#"],[0,0,0,"#","#","#","#"],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,"$","$","$","$"],[0,0,0,"$","$","$",1],[0,0,0,"$","$",1,"#"],[0,0,0,"$",1,"#","#"],[0,0,0,1,"#","#","#"],[0,0,0,"#","#","#","#"]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,"$","$","$","$"],[0,0,0,"$","$","$",1],[0,0,0,"$","$",1,"#"],[0,0,0,"$",1,"#","#"],[0,0,0,1,"#","#","#"]])
        #Horizontal check
        mask.append([[1,1,1,1,0,0,0],["#","#","#","#",0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([["$","$","$","$",0,0,0],[1,1,1,1,0,0,0],["#","#","#","#",0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],["$","$","$","$",0,0,0],[1,1,1,1,0,0,0],["#","#","#","#",0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],["$","$","$","$",0,0,0],[1,1,1,1,0,0,0],["#","#","#","#",0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],["$","$","$","$",0,0,0],[1,1,1,1,0,0,0],["#","#","#","#",0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],["$","$","$","$",0,0,0],[1,1,1,1,0,0,0]])
        mask.append([[0,1,1,1,1,0,0],[0,"#","#","#","#",0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,"$","$","$","$",0,0],[0,1,1,1,1,0,0],[0,"#","#","#","#",0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,"$","$","$","$",0,0],[0,1,1,1,1,0,0],[0,"#","#","#","#",0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,"$","$","$","$",0,0],[0,1,1,1,1,0,0],[0,"#","#","#","#",0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,"$","$","$","$",0,0],[0,1,1,1,1,0,0],[0,"#","#","#","#",0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,"$","$","$","$",0,0],[0,1,1,1,1,0,0]])
        mask.append([[0,0,1,1,1,1,0],[0,0,"#","#","#","#",0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,"$","$","$","$",0],[0,0,1,1,1,1,0],[0,0,"#","#","#","#",0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,"$","$","$","$",0],[0,0,1,1,1,1,0],[0,0,"#","#","#","#",0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,"$","$","$","$",0],[0,0,1,1,1,1,0],[0,0,"#","#","#","#",0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,"$","$","$","$",0],[0,0,1,1,1,1,0],[0,0,"#","#","#","#",0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,"$","$","$","$",0],[0,0,1,1,1,1,0]])
        mask.append([[0,0,0,1,1,1,1],[0,0,0,"#","#","#","#"],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,"$","$","$","$"],[0,0,0,1,1,1,1],[0,0,0,"#","#","#","#"],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,"$","$","$","$"],[0,0,0,1,1,1,1],[0,0,0,"#","#","#","#"],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,"$","$","$","$"],[0,0,0,1,1,1,1],[0,0,0,"#","#","#","#"],[0,0,0,0,0,0,0]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,"$","$","$","$"],[0,0,0,1,1,1,1],[0,0,0,"#","#","#","#"]])
        mask.append([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,"$","$","$","$"],[0,0,0,1,1,1,1]])
        #vertical
        #mask.append([[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],["#",0,0,0,0,0,0],["#",0,0,0,0,0,0]])
        #mask.append([["$",0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],["#",0,0,0,0,0,0]])
        #mask.append([["$",0,0,0,0,0,0],["$",0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0],[1,0,0,0,0,0,0]])
        #mask.append([[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,"#",0,0,0,0,0],[0,"#",0,0,0,0,0]])
        #mask.append([[0,"$",0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,"#",0,0,0,0,0]])
        #mask.append([[0,"$",0,0,0,0,0],[0,"$",0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0],[0,1,0,0,0,0,0]])
        #mask.append([[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,"#",0,0,0,0],[0,0,"#",0,0,0,0]])
        #mask.append([[0,0,"$",0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,"#",0,0,0,0]])
        #mask.append([[0,0,"$",0,0,0,0],[0,0,"$",0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0]])
        #mask.append([[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,"#",0,0,0],[0,0,0,"#",0,0,0]])
        #mask.append([[0,0,0,"$",0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,"#",0,0,0]])
        #mask.append([[0,0,0,"$",0,0,0],[0,0,0,"$",0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,0,0,0]])
        #mask.append([[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,"#",0,0],[0,0,0,0,"#",0,0]])
        #mask.append([[0,0,0,0,"$",0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,"#",0,0]])
        #mask.append([[0,0,0,0,"$",0,0],[0,0,0,0,"$",0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,1,0,0]])
        #mask.append([[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,"#",0],[0,0,0,0,0,"#",0]])
        #mask.append([[0,0,0,0,0,"$",0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,"#",0]])
        #mask.append([[0,0,0,0,0,"$",0],[0,0,0,0,0,"$",0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0],[0,0,0,0,0,1,0]])
        #mask.append([[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,"#"],[0,0,0,0,0,0,"#"]])
        #mask.append([[0,0,0,0,0,0,"$"],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,"#"]])
        #mask.append([[0,0,0,0,0,0,"$"],[0,0,0,0,0,0,"$"],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1]])
        
        len_curr = len(mask)
        #print("Length of mask as of now:", len_curr)
        ## Some additional code goes here ##
        #for i in range(len_curr):
        #    temp = self.transpose(mask[i])
        #    mask.append(temp)
        #print("Now the length of mask is", len(mask))
        return mask

